In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Feature dataset so we will remove label ie target ie price
submission_test = pd.read_excel('Data_Test.xlsx')
train = pd.read_excel('Data_Train.xlsx')

In [3]:
train.iloc[:,:-1]

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts"
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery"
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography"
...,...,...,...,...,...,...,...,...
6232,Humans: A Brief History of How We F*cked It Al...,Tom Phillips,"Paperback,– 8 Aug 2018",5.0 out of 5 stars,2 customer reviews,'F*cking brilliant' Sarah Knight\n'Very funny'...,Anthropology (Books),Humour
6233,The Chemist,Stephenie Meyer,"Paperback,– 21 Nov 2016",3.3 out of 5 stars,9 customer reviews,"In this gripping page-turner, an ex-agent on t...",Contemporary Fiction (Books),"Crime, Thriller & Mystery"
6234,The Duke And I: Number 1 in series (Bridgerton...,Julia Quinn,"Paperback,– 8 Jun 2006",3.8 out of 5 stars,3 customer reviews,'The most refreshing and radiant love story yo...,Romance (Books),Romance
6235,Frostfire (Kanin Chronicles),Amanda Hocking,"Paperback,– 15 Jan 2015",3.5 out of 5 stars,4 customer reviews,Frostfire by Amanda Hocking is the stunning fi...,Action & Adventure (Books),Action & Adventure


In [5]:
print('train shape: ', train.shape)

train shape:  (6237, 9)


In [6]:
df = pd.concat([train.iloc[:,:-1],test])
df.head()

NameError: name 'test' is not defined

# PIPELINE

In [8]:
import re

In [9]:
def data_pipeline(df):
    
    df['Publish_year'] = df['Edition'].apply(lambda x: x.split(" ")[-1])
    # since there were few rows with text in publish year so we first removed it then convert them into float, int
    df = df[pd.to_numeric(df['Publish_year'], errors = 'coerce').notnull()]
    
    
    df['no_of_rating'] = df['Ratings'].apply(lambda x: (x.split(" ")[0]).replace(",",""))
    df['Author_count'] = np.zeros
    
    for i in range(len(df)):
        df['Author_count'].iloc[i] = (df['Author'].iloc[i].count(",") + 1)
        df['Reviews'] = df['Reviews'].apply(lambda x: x.split(" ")[0])
        


    df['Book_type'] = df['Edition'].apply(lambda x: x.split(",")[0])

    df['Author_count'] = df['Author_count'].astype(int)
    df['Reviews'] = df['Reviews'].astype(float)
    df['no_of_rating'] = df['no_of_rating'].astype(int)
    df['Publish_year'] = df['Publish_year'].astype(int)

    
    # Creating book categories 
    
    book_cat = df['BookCategory'].apply(lambda x: re.split(' & |, ', x))
    book_cat_dict = {}
    for i in range(len(book_cat)):
        for val in book_cat.iloc[i]:
            if val in book_cat_dict:
                book_cat_dict[val] +=1
            else:
                book_cat_dict[val] = 0

    category_df = pd.DataFrame(index = range(len(df)), columns = book_cat_dict.keys())
    category_df.fillna(0, inplace=True)
    
    for i in range(len(df)):
        for val in book_cat.iloc[i]:
            category_df[val].iloc[i] = 1
    


    df = pd.concat([df.reset_index(drop=True), category_df.reset_index(drop=True)], axis=1)
    
    
    df = pd.get_dummies(df,columns = ['Book_type'])
    # returning the values
    
    

    bookname_df = df['Title']
    try:
        y = df['Price']
    except:
        y = pd.DataFrame(index = range(len(df)), columns = ['Price'])
    
    try:
        df.drop(['Title', 'Author', 'Edition', 'Ratings', 'Synopsis', 'Genre', 'BookCategory', 'Price'],axis=1, inplace=True)    
        X = df
    except:
        df.drop(['Title', 'Author', 'Edition', 'Ratings', 'Synopsis', 'Genre', 'BookCategory'],axis=1, inplace=True)    
        X = df
        
    
    return X,y, bookname_df

In [10]:
train = pd.read_excel('Data_Train.xlsx')
X,y,bookname_df = data_pipeline(train)

<ipython-input-9-732b4c489f02>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['no_of_rating'] = df['Ratings'].apply(lambda x: (x.split(" ")[0]).replace(",",""))
<ipython-input-9-732b4c489f02>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Author_count'] = np.zeros
C:\Users\Axelrod\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

In [11]:
X['Publish_year'].value_counts(ascending = False)[-30:]

1996    27
1993    22
1989    20
1991    19
1986    16
1984    15
1988    15
1982    14
1990    14
1987    13
1985    10
1983    10
1976     6
1980     5
1971     4
1975     4
1977     3
1964     3
1974     3
1978     2
1979     2
1960     2
1900     1
1925     1
1961     1
1969     1
1905     1
1970     1
1981     1
1973     1
Name: Publish_year, dtype: int64

In [12]:
X

,Reviews,Publish_year,no_of_rating,Author_count,Action,Adventure,Biographies,Diaries,True Accounts,Humour,...,Book_type_Leather Bound,Book_type_Library Binding,Book_type_Loose Leaf,Book_type_Mass Market Paperback,Book_type_Paperback,Book_type_Perfect Paperback,Book_type_Product Bundle,Book_type_Sheet music,Book_type_Spiral-bound,Book_type_Tankobon Softcover
0,4.0,2016,8,1,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,3.9,2012,14,1,0,0,1,1,1,0,...,0,0,0,0,1,0,0,0,0,0
2,4.8,1982,6,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,4.1,2017,13,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5.0,2006,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6211,5.0,2018,2,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
6212,3.3,2016,9,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6213,3.8,2006,3,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6214,3.5,2015,4,1,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
X.shape

(6216, 45)

In [14]:
y

0       220.00
1       202.93
2       299.00
3       180.00
4       965.62
         ...  
6211    322.00
6212    421.00
6213    399.00
6214    319.00
6215    452.00
Name: Price, Length: 6216, dtype: float64

In [15]:
bookname_df

0                     The Prisoner's Gold (The Hunters 3)
1                      Guru Dutt: A Tragedy in Three Acts
2                            Leviathan (Penguin Classics)
3                      A Pocket Full of Rye (Miss Marple)
4              LIFE 70 Years of Extraordinary Photography
                              ...                        
6211    Humans: A Brief History of How We F*cked It Al...
6212                                          The Chemist
6213    The Duke And I: Number 1 in series (Bridgerton...
6214                         Frostfire (Kanin Chronicles)
6215                          The First Order (Sam Capra)
Name: Title, Length: 6216, dtype: object

# Model Fitting

In [16]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [17]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [18]:
# predicting price of the submission data
Xs, ys, bookname_dfs = data_pipeline(submission_test)

<ipython-input-9-732b4c489f02>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['no_of_rating'] = df['Ratings'].apply(lambda x: (x.split(" ")[0]).replace(",",""))
<ipython-input-9-732b4c489f02>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Author_count'] = np.zeros
C:\Users\Axelrod\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

In [19]:
from sklearn.metrics import mean_squared_error
predictions = lin_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, predictions)
line_rmse = np.sqrt(lin_mse)
print(line_rmse)

499.77305372823656


In [20]:
print('Predictions: ', lin_reg.predict(X_test[:5]))
print('\n Original values: ', list(y_test[:5]))


Predictions:  [436.3943723  855.47937411 304.51425631 669.51056503 733.20259184]

 Original values:  [320.0, 472.0, 541.37, 247.0, 309.0]


In [21]:
print('Predictions: ', lin_reg.predict(X_train[:5]))
print('\n Original values: ', list(y_train[:5]))


Predictions:  [299.09132284 393.06003784 189.30414838 322.88018043 856.29275346]

 Original values:  [314.0, 295.0, 239.0, 375.0, 456.0]


In [22]:
y_test

3591    320.00
5254    472.00
5734    541.37
3048    247.00
185     309.00
         ...  
3369    374.00
62      247.00
1932    571.00
4457    499.00
2717    227.00
Name: Price, Length: 1244, dtype: float64